# Purpose

Run Vectorizing for all NEW subreddits in v0.4.1 

---

This notebook runs the `vectorize_text_to_embeddings` function to:
- loading USE-multilingual model
- load post & comment text
- convert the text into embeddings (at post or comment level)


# Notebook setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import gc
# from functools import partial
# import os
import logging
# from pathlib import Path
# from pprint import pprint

import mlflow

# from tqdm.auto import tqdm
from tqdm import tqdm
import numpy as np
import pandas as pd

from google.cloud import storage

# TF libraries... I've been getting errors when these aren't loaded
import tensorflow_text
import tensorflow as tf

import subclu
from subclu.utils.hydra_config_loader import LoadHydraConfig
from subclu.models.vectorize_text import (
    vectorize_text_to_embeddings,
)
from subclu.models import vectorize_text_tf

from subclu.utils import set_working_directory
from subclu.utils.mlflow_logger import MlflowLogger
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric
)


print_lib_versions([mlflow, np, mlflow, pd, tensorflow_text, tf, subclu])

python		v 3.7.10
===
mlflow		v: 1.16.0
numpy		v: 1.18.5
mlflow		v: 1.16.0
pandas		v: 1.2.5
tensorflow_text	v: 2.3.0
tensorflow	v: 2.3.3
subclu		v: 0.4.1


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Initialize mlflow logging with sqlite database

In [4]:
# use new class to initialize mlflow
mlf = MlflowLogger(tracking_uri='sqlite')
mlflow.get_tracking_uri()

'sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db'

## Get list of experiments with new function

In [5]:
mlf.list_experiment_meta(output_format='pandas')

,experiment_id,name,artifact_location,lifecycle_stage
0,0,Default,./mlruns/0,active
1,1,fse_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/1,active
2,2,fse_vectorize_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/2,active
3,3,subreddit_description_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/3,active
4,4,fse_vectorize_v1.1,gs://i18n-subreddit-clustering/mlflow/mlruns/4,active
5,5,use_multilingual_v0.1_test,gs://i18n-subreddit-clustering/mlflow/mlruns/5,active
6,6,use_multilingual_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/6,active
7,7,use_multilingual_v1_aggregates_test,gs://i18n-subreddit-clustering/mlflow/mlruns/7,active
8,8,use_multilingual_v1_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/8,active
9,9,v0.3.2_use_multi_inference_test,gs://i18n-subreddit-clustering/mlflow/mlruns/9,active


# Check whether we have access to a GPU

In [6]:
l_phys_gpus = tf.config.list_physical_devices('GPU')
# from tensorflow.python.client import device_lib

print(
    f"\nBuilt with CUDA? {tf.test.is_built_with_cuda()}"
    f"\nGPUs\n==="
    f"\nNum GPUs Available: {len(l_phys_gpus)}"
    f"\nGPU details:\n{l_phys_gpus}"
#     f"\n\nAll devices:\n===\n"
#     f"{device_lib.list_local_devices()}"
)


Built with CUDA? True
GPUs
===
Num GPUs Available: 1
GPU details:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Load config with data to process

In [7]:
config_data_v041 = LoadHydraConfig(
    config_path="../config/data_text_and_metadata",
    config_name='v0.4.1_2021_12',
)
config_data_v041.config_dict

{'dataset_name': 'v0.4.1 inputs - 50k subreddits - Active Subreddits (no Geo) + Geo-relevant users_l28>=100 & posts_l28>=4',
 'bucket_name': 'i18n-subreddit-clustering',
 'folder_subreddits_text_and_meta': 'subreddits/top/2021-12-14',
 'folder_posts_text_and_meta': 'posts/top/2021-12-14',
 'folder_comments_text_and_meta': 'comments/top/2021-12-14',
 'folder_subreddits_text_and_meta_filter': 'subreddits/top/2021-09-24'}

In [8]:
mlflow_experiment_test = 'v0.4.1_mUSE_inference_test'
mlflow_experiment_full = 'v0.4.1_mUSE_inference'

bucket_name = config_data_v041.config_dict['bucket_name']
subreddits_path = config_data_v041.config_dict['folder_subreddits_text_and_meta']
posts_path = config_data_v041.config_dict['folder_posts_text_and_meta']

subreddits_path_exclude = config_data_v041.config_dict['folder_subreddits_text_and_meta_filter']
# comments_path = None

# Side bar - had to move files 
## from subfolder `2021-12-14` to: `2021-12-24`...
dislexia strikes again

```python
# check list of files
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
l_comment_files_to_process = list(bucket.list_blobs(prefix=subreddits_path))
total_comms_file_count = len(l_comment_files_to_process)
print(total_comms_file_count)

new_folder = "gs://i18n-subreddit-clustering/posts/top/2021-12-14/"
for blob_ in list(bucket.list_blobs(prefix='posts/top/2021-12-24')):
    current_name = f"gs://i18n-subreddit-clustering/{blob_.name}"
    
    `!gsutil mv $current_name   $new_folder`  # remove tick marks, in actual code, only used them to fix display issue
    
# > Copying gs://i18n-subreddit-clustering/posts/top/2021-12-24/000000000001.parquet [Content-Type=application/octet-stream]...
# > Removing gs://i18n-subreddit-clustering/posts/top/2021-12-24/000000000001.parquet...

```

# Run test on sample files
Only one or two files so that we catch some errors before running on whole data set

In [71]:
mlflow.end_run(status='KILLED')

vectorize_text_tf.vectorize_text_to_embeddings(
    model_name='use_multilingual',
    run_name=f"posts_as_comments_batch_fxn-{datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')}",
    mlflow_experiment=mlflow_experiment_full,
    
    tokenize_lowercase=False,
    # subreddits_path_exclude=subreddits_path_exclude,  # New param to exclude embeddings in these subs
    batch_comment_files=True,
    
    # Check 17 b/c it seems to have a duplicated post
    n_comment_files_slice_start=17,
    n_comment_files_slice_end=18,

    bucket_name=bucket_name,
    subreddits_path=None,  # subreddits_path,
    posts_path=None,  # posts_path
    comments_path=posts_path,

    # Hack: Rename cols so that I can process posts as a batch of comments
    col_post_id=None,
    col_comment_id='post_id',
    col_text_comment='text',
    col_text_comment_word_count='text_word_count',
    cols_index_comment=['subreddit_name', 'subreddit_id', 'post_id'],
    local_comms_subfolder_relative='df_vect_posts',
    mlflow_comments_folder='df_vect_posts',
    
    tf_batch_inference_rows=2600,
    tf_limit_first_n_chars=850,
    
    n_sample_comment_files=None,
    
#     n_sample_posts=9500,
#     n_sample_comments=19100,
)

23:27:18 | INFO | "Start vectorize function"
23:27:18 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-12-17_232718"
23:27:18 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db"
23:27:19 | INFO | "host_name: djb-subclu-inference-tf-2-3-20210630"
23:27:19 | INFO | "  Saving config to local path..."
23:27:19 | INFO | "  Logging config to mlflow with joblib..."
23:27:19 | INFO | "  Logging config to mlflow with YAML..."
23:27:20 | INFO | "Loading model use_multilingual..."


23:27:22 | INFO | "  0:00:02.358327 <- Load TF HUB model time elapsed"
23:27:22 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
23:27:22 | INFO | "** Procesing Comments files one at a time ***"
23:27:22 | INFO | "-- Loading & vectorizing COMMENTS in files: 40 --
Expected batch size: 2600"
23:27:22 | WARNING | "df_posts missing, so we can't filter comments without a post...
local variable 'df_posts' referenced before assignment"
  0%|          | 0/40 [00:00<?, ?it/s]23:27:22 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000000.parquet --"
23:27:22 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000001.parquet --"
23:27:22 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000002.parquet --"
23:27:22 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000003.parquet --"
23:27:22 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000004.parquet --"
23:27:22 | INFO | "    -- Skipping file: posts/top/2021-12-14/00

23:27:23 | INFO | "Processing: posts/top/2021-12-14/000000000017.parquet"
23:27:25 | WARNING | "Found duplicate IDs in col: post_id"
23:27:25 | INFO | "Keeping only one row_per ID"
23:27:25 | INFO | "  (350847, 5) <- df_comments.shape AFTER removing duplicates"
23:27:26 | INFO | "Getting embeddings in batches of size: 2600"
  0%|                                                   | 0/135 [00:00<?, ?it/s]

100%|#########################################| 135/135 [02:03<00:00,  1.10it/s]
23:29:30 | INFO | "  Saving to local: df_vect_posts/000000000017 | 350,847 Rows by 515 Cols"
 45%|####5     | 18/40 [02:18<02:49,  7.71s/it]23:29:41 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000018.parquet --"
23:29:41 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000019.parquet --"
23:29:41 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000020.parquet --"
23:29:41 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000021.parquet --"
23:29:41 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000022.parquet --"
23:29:41 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000023.parquet --"
23:29:41 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000024.parquet --"
23:29:41 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000025.parquet --"
23:29:41 | INFO | "    -- Skipping file: posts/top/2021-12-14/000000000026.parquet 

In [9]:
gc.collect()

6091

# Run full with `BATCHING` - `lower_case=False`

60GB of RAM wasn't good enough for 19Million comments _lol_... so let's just do batching by default.


Error without batching (loading all parquet files into a single file):
```
...
12:02:14 | INFO | "  (19168154, 6) <- updated df_comments shape"
12:02:14 | INFO | "Vectorizing COMMENTS..."
12:02:14 | INFO | "Getting embeddings in batches of size: 2100"
100%
9128/9128 [1:32:26<00:00, 1.97it/s]

<__array_function__ internals> in concatenate(*args, **kwargs)

MemoryError: Unable to allocate 36.6 GiB for an array with shape (512, 19168154) and data type float32
```


In [77]:
comments_path = None
print(mlflow_experiment_test)
print(posts_path)

v0.4.1_mUSE_inference_test
posts/top/2021-12-14


In [ ]:
mlflow.end_run(status='KILLED')

vectorize_text_tf.vectorize_text_to_embeddings(
    model_name='use_multilingual',
    run_name=f"posts_as_comments_new_fxn_no_filters-{datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')}",
    mlflow_experiment=mlflow_experiment_full,
    
    tokenize_lowercase=False,
    batch_comment_files=True,
    
    bucket_name=bucket_name,
    subreddits_path=subreddits_path,  # subreddits_path,
    posts_path=None,  # posts_path
    comments_path=posts_path,
    subreddits_path_exclude=subreddits_path_exclude,  # New param to exclude embeddings in these subs

    # Hack: Rename cols so that I can process posts as a batch of comments
    col_post_id=None,
    col_comment_id='post_id',
    col_text_comment='text',
    col_text_comment_word_count='text_word_count',
    cols_index_comment=['subreddit_name', 'subreddit_id', 'post_id'],
    local_comms_subfolder_relative='df_vect_posts',
    mlflow_comments_folder='df_vect_posts',
    
    tf_batch_inference_rows=2500,
    tf_limit_first_n_chars=850,

)

23:56:27 | INFO | "Start vectorize function"
23:56:27 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-12-17_235627"
23:56:27 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db"
23:56:27 | INFO | "host_name: djb-subclu-inference-tf-2-3-20210630"
23:56:27 | INFO | "  Saving config to local path..."
23:56:27 | INFO | "  Logging config to mlflow with joblib..."
23:56:28 | INFO | "  Logging config to mlflow with YAML..."
23:56:28 | INFO | "Loading model use_multilingual..."


23:56:30 | INFO | "  0:00:02.280445 <- Load TF HUB model time elapsed"
23:56:30 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
23:56:30 | INFO | "Load subreddits df..."
23:56:31 | INFO | "  0:00:00.684703 <- df_subs_exclude loading time elapsed"
23:56:31 | INFO | "  (19262, 4) <- df_subs_exclude shape"
23:56:31 | INFO | "Load subreddits df..."
23:56:32 | INFO | "  0:00:00.768617 <- df_subs loading time elapsed"
23:56:32 | INFO | "  (49705, 4) <- df_subs shape"
23:56:32 | INFO | "Vectorizing subreddit descriptions..."


23:56:33 | INFO | "Getting embeddings in batches of size: 2500"
  0%|                                                    | 0/20 [00:00<?, ?it/s]

100%|###########################################| 20/20 [00:28<00:00,  1.41s/it]
23:57:01 | INFO | "  0:00:29.216174 <- df_subs vectorizing time elapsed"
23:57:01 | INFO | "  Saving to local: df_vect_subreddits_description/df | 49,705 Rows by 514 Cols"
23:57:01 | INFO | "Converting pandas to dask..."
23:57:01 | INFO | "   103.4 MB <- Memory usage"
23:57:01 | INFO | "       2	<- target Dask partitions	  100.0 <- target MB partition size"
23:57:02 | INFO | "  Logging to mlflow..."
23:57:04 | INFO | "** Procesing Comments files one at a time ***"
23:57:05 | INFO | "-- Loading & vectorizing COMMENTS in files: 40 --
Expected batch size: 2500"
23:57:05 | WARNING | "df_posts missing, so we can't filter comments without a post...
local variable 'df_posts' referenced before assignment"
  0%|          | 0/40 [00:00<?, ?it/s]23:57:05 | INFO | "Processing: posts/top/2021-12-14/000000000000.parquet"
23:57:07 | WARNING | "Found duplicate IDs in col: post_id"
23:57:07 | INFO | "Keeping only one row_p

In [13]:
mlflow.end_run(status='KILLED')

## Run it on all posts, not just the posts for new files

In [ ]:
mlflow.end_run(status='KILLED')

vectorize_text_tf.vectorize_text_to_embeddings(
    model_name='use_multilingual',
    run_name=f"posts_as_comments_new_fxn_no_filters-{datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')}",
    mlflow_experiment=mlflow_experiment_full,
    
    tokenize_lowercase=False,
    batch_comment_files=True,
    
    bucket_name=bucket_name,
    subreddits_path=subreddits_path,  # subreddits_path,
    posts_path=None,  # posts_path
    comments_path=posts_path,

    # Hack: Rename cols so that I can process posts as a batch of comments
    col_post_id=None,
    col_comment_id='post_id',
    col_text_comment='text',
    col_text_comment_word_count='text_word_count',
    cols_index_comment=['subreddit_name', 'subreddit_id', 'post_id'],
    local_comms_subfolder_relative='df_vect_posts',
    mlflow_comments_folder='df_vect_posts',
    
    tf_batch_inference_rows=2500,
    tf_limit_first_n_chars=850,

)

04:48:53 | INFO | "Start vectorize function"
04:48:53 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-12-18_044853"
04:48:54 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db"
04:48:54 | INFO | "host_name: djb-subclu-inference-tf-2-3-20210630"
04:48:54 | INFO | "  Saving config to local path..."
04:48:54 | INFO | "  Logging config to mlflow with joblib..."
04:48:55 | INFO | "  Logging config to mlflow with YAML..."
04:48:55 | INFO | "Loading model use_multilingual..."
04:48:57 | INFO | "  0:00:02.248069 <- Load TF HUB model time elapsed"
04:48:57 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
04:48:57 | INFO | "Load subreddits df..."
04:48:58 | INFO | "  0:00:00.786374 <- df_subs loading time elapsed"
04:48:58 | INFO | "  (49705, 4) <- df_subs shape"
04:48:58 | INFO | "Vectorizing subreddit descript

### Re-run comments and log to non-test mlflow experiment


Besides file-batching, this job increased the row-batches from 2,000 to 6,100... unclear if this is having a negative impact. Maybe smaller batches are somehow more efficient?
Now that I'm reading one file at a time, it looks like speed is taking a big hit

Baseline when running it all in memory. It took `1:32:26`, but it ran out of memory (RAM).
The current ETA is around `2 hours`

```
# singe file, all in memory (results in OOM)
12:02:14 | INFO | "Vectorizing COMMENTS..."
12:02:14 | INFO | "Getting embeddings in batches of size: 2100"
100%
9128/9128 [1:32:26<00:00, 1.97it/s]


# one file at a time... slower, but we get results one file at a time...
16%
6/37 [21:11<1:49:46, 212.45s/it]
```


# Run full with `lower_case=True`

This one is expected to be a little slower because it'll call `.str.lower()` on each batch of text.

---

TODO: unsure if it's worth running this job in parallel while I do work on a separate VM... might be a big pain to manually sync the rows from metrics & params happening at the same time in two different VMs.



In [26]:
mlflow.end_run(status='KILLED')

vectorize_text_tf.vectorize_text_to_embeddings(
    model_name='use_multilingual',
    run_name=f"posts_as_comments_batch_fxn-{datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')}",
    mlflow_experiment=mlflow_experiment_full,
    
    tokenize_lowercase=True,
    
    bucket_name=bucket_name,
    subreddits_path=subreddits_path,
    posts_path=None,  # posts_path
    comments_path=posts_path,

    # Hack: Rename cols so that I can process posts as a batch of comments
    col_post_id=None,
    col_comment_id='post_id',
    col_text_comment='text',
    col_text_comment_word_count='text_word_count',
    cols_index_comment=['subreddit_name', 'subreddit_id', 'post_id'],
    local_comms_subfolder_relative='df_vect_posts',
    mlflow_comments_folder='df_vect_posts',
    
    tf_batch_inference_rows=2600,
    tf_limit_first_n_chars=850,
    
    n_sample_comment_files=None,
    
#     n_sample_posts=9500,
#     n_sample_comments=19100,
)

10:05:02 | INFO | "Start vectorize function"
10:05:02 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-09-28_100502"
10:05:02 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db"
10:05:03 | INFO | "  Saving config to local path..."
10:05:03 | INFO | "  Logging config to mlflow..."
10:05:03 | INFO | "Loading model use_multilingual..."
10:05:05 | INFO | "  0:00:02.265257 <- Load TF HUB model time elapsed"
10:05:05 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
10:05:05 | INFO | "Load subreddits df..."
10:05:06 | INFO | "  0:00:00.683829 <- df_subs loading time elapsed"
10:05:06 | INFO | "  (19262, 4) <- df_subs shape"
10:05:06 | INFO | "Vectorizing subreddit descriptions..."
10:05:06 | INFO | "Getting embeddings in batches of size: 2600"
  0%|                                                     | 0/8 [00

# Debug - why are there duplicates in file 17?

Looks like the reason for the duplicate is that there's at least one post where the OCR text is different... unclear why we have two separate rows, but maybe we can drop the dupe and keep the post wit the longest text.


In [52]:
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
l_comment_files_to_process = list(bucket.list_blobs(prefix=posts_path))
total_comms_file_count = len(l_comment_files_to_process)
print(total_comms_file_count)

new_folder = "gs://i18n-subreddit-clustering/posts/top/2021-12-14/"
for blob_ in l_comment_files_to_process:
    current_name = f"gs://i18n-subreddit-clustering/{blob_.name}"

40


In [63]:
%%time
l_cols_ix_posts = [
    'subreddit_name',
    'subreddit_id',
    'post_id',
]
l_cols_text = [
    'text',
    'weighted_language',
    'weighted_language_probability',
]

df_17 = pd.read_parquet(
    f"gs://i18n-subreddit-clustering/posts/top/2021-12-14/000000000017.parquet",
    # columns=l_cols_ix_posts + l_cols_text,
)
df_17.shape

CPU times: user 1.43 s, sys: 335 ms, total: 1.76 s
Wall time: 2.86 s


(350848, 44)

In [64]:
counts_describe(df_17[l_cols_ix_posts])

,dtype,count,unique,unique-percent,null-count,null-percent
subreddit_name,object,"350,848","1,102",0.31%,0,0.00%
subreddit_id,object,"350,848","1,102",0.31%,0,0.00%
post_id,object,"350,848","350,847",100.00%,0,0.00%


In [65]:
df_17[df_17.duplicated(subset=l_cols_ix_posts, keep=False)]

,rank_post_in_sub,subreddit_name,subreddit_id,post_id,user_id,submit_date,removed,upvotes,upvotes_lookup,downvotes_lookup,net_upvotes_lookup,neutered,verdict,content_category,comments,successful,app_name,post_type,post_nsfw,post_url,geolocation_country_code,rating_short,rating_name,primary_topic,geo_relevant_countries,geo_relevant_country_codes,geo_relevant_subreddit,geo_relevant_subreddit_all,geo_relevant_subreddit_v04,ambassador_or_default_any,ambassador_or_default_sub_france,ambassador_or_default_sub_germany,weighted_language,weighted_language_probability,post_language_preference,ocr_images_in_post_count,flair_text,text_len,ocr_text_len,text_word_count,ocr_text_word_count,post_url_for_embeddings,text,ocr_inferred_text_agg_clean
62218,602,caguns,t5_31iwc,t3_rdma9j,t2_cugrenid,2021-12-10,0,6,8,3,5,False,None,None,11,1,web2x,multi_media,False,/r/CAguns/comments/rdma9j/the_journey_is_complete/,us,E,Everyone,Hobbies,None,None,False,None,False,None,None,None,en,0.999996,en,1.0,None,3971,37.0,745,6.0,None,"The journey is complete! Right around the beginning of September, being a new gun owner in CA, I decided that I wanted a 1911 handgun. But not just any 1911 - it *had* to be a Springfield 1911 Loaded Marine Corps Operator. But even more...",SPRINGFIELD ARMORY SI GENESEO IL USA
62219,602,caguns,t5_31iwc,t3_rdma9j,t2_cugrenid,2021-12-10,0,6,8,3,5,False,None,None,11,1,web2x,multi_media,False,/r/CAguns/comments/rdma9j/the_journey_is_complete/,us,E,Everyone,Hobbies,None,None,False,None,False,None,None,None,en,0.999996,en,1.0,None,3971,27.0,745,5.0,None,"The journey is complete! Right around the beginning of September, being a new gun owner in CA, I decided that I wanted a 1911 handgun. But not just any 1911 - it *had* to be a Springfield 1911 Loaded Marine Corps Operator. But even more...",USA H USAH LOADED OPERATOR


In [66]:
df_17[df_17.duplicated(keep=False)]

,rank_post_in_sub,subreddit_name,subreddit_id,post_id,user_id,submit_date,removed,upvotes,upvotes_lookup,downvotes_lookup,net_upvotes_lookup,neutered,verdict,content_category,comments,successful,app_name,post_type,post_nsfw,post_url,geolocation_country_code,rating_short,rating_name,primary_topic,geo_relevant_countries,geo_relevant_country_codes,geo_relevant_subreddit,geo_relevant_subreddit_all,geo_relevant_subreddit_v04,ambassador_or_default_any,ambassador_or_default_sub_france,ambassador_or_default_sub_germany,weighted_language,weighted_language_probability,post_language_preference,ocr_images_in_post_count,flair_text,text_len,ocr_text_len,text_word_count,ocr_text_word_count,post_url_for_embeddings,text,ocr_inferred_text_agg_clean


In [ ]:
LEGACY

# Run full with lower_case=False

Time on CPU, only comments + subs:
```
13:29:07 | INFO | "  (1108757, 6) <- df_comments shape"
13:29:08 | INFO | "  (629, 4) <- df_subs shape"

13:45:11 | INFO | "  0:16:21.475036 <- Total vectorize fxn time elapsed"
```

In [18]:
mlflow.end_run(status='KILLED')

model, df_vect, df_vect_comments, df_vect_subs = vectorize_text_to_embeddings(
    model_name='use_multilingual',
    run_name='full_data-lowercase_false',
    mlflow_experiment=mlflow_experiment_full,
    
    tokenize_lowercase=False,
    subreddits_path='subreddits/de/2021-06-16',
    posts_path=None,  # 'posts/de/2021-06-16',
    comments_path='comments/de/2021-06-16',
    tf_batch_inference_rows=1500,
    tf_limit_first_n_chars=1100,
    n_sample_posts=None,
    n_sample_comments=None,
)

13:28:50 | INFO | "Start vectorize function"
13:28:50 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-07-01_1328"
13:28:50 | INFO | "Load comments df..."
13:29:07 | INFO | "  (1108757, 6) <- df_comments shape"
13:29:07 | INFO | "Keep only comments that match posts IDs in df_posts..."
13:29:07 | INFO | "df_posts missing, so we can't filter comments..."
13:29:07 | INFO | "Load subreddits df..."
13:29:08 | INFO | "  (629, 4) <- df_subs shape"
13:29:08 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/mlflow/mlruns.db"
13:29:09 | INFO | "Loading model use_multilingual...
  with kwargs: None"


13:29:11 | INFO | "  0:00:02.282361 <- Load TF HUB model time elapsed"
13:29:11 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
13:29:11 | INFO | "Vectorizing subreddit descriptions..."


13:29:13 | INFO | "  Saving to local... df_vect_subreddits_description..."
13:29:13 | INFO | "  Logging to mlflow..."


13:29:14 | INFO | "Vectorizing COMMENTS..."
13:29:14 | INFO | "Getting embeddings in batches of size: 1500"


  0%|          | 0/740 [00:00<?, ?it/s]

13:44:30 | INFO | "  Saving to local... df_vect_comments..."
13:44:49 | INFO | "  Logging to mlflow..."
13:45:11 | INFO | "  0:16:21.475036 <- Total vectorize fxn time elapsed"
